In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 100     # number of variables
num_ineq = 100    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("gurobi")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_exact_100-100.csv")
print("Number of None values: ", df["Sol"].isna().sum())

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_100-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:23<00:00,  2.60it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -21.886359          5.596299      0.173362
std       0.462866          1.133289      0.011438
min     -23.293625          2.305694      0.157268
25%     -22.198093          4.785628      0.166839
50%     -21.885911          5.557468      0.170520
75%     -21.562072          6.319377      0.176169
max     -20.401462          9.460674      0.245447
Number of infeasible solution: 1000


In [8]:
df = pd.read_csv("result/cq_heur_rnd_100-100.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -21.922316          5.342438      0.169697
std      0.454859          1.039560      0.009731
min    -22.983994          2.388443      0.157866
25%    -22.220240          4.613285      0.163289
50%    -21.891719          5.409654      0.168302
75%    -21.608674          6.064012      0.173709
max    -20.763606          8.426283      0.223535
Number of infeasible solution: 100


## Heuristic - N1

In [9]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n1_100-100.csv")

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [11:55:24<00:00, 42.92s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.587003e+18               0.0     42.896583
std    5.831904e+17               0.0     16.178696
min   -1.834646e+01               0.0      4.436790
25%    1.193007e+18               0.0     30.471017
50%    1.486973e+18               0.0     41.963484
75%    1.871542e+18               0.0     54.495735
max    4.563401e+18               0.0    102.884029
Number of infeasible solution: 0


In [9]:
df = pd.read_csv("result/cq_heur_n1_100-100.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02             100.0    100.000000
mean   1.687085e+18               0.0     43.938824
std    5.642133e+17               0.0     17.129032
min    8.294812e+17               0.0     14.204978
25%    1.296812e+18               0.0     32.587548
50%    1.503969e+18               0.0     43.402815
75%    1.932558e+18               0.0     55.671246
max    3.480013e+18               0.0    102.884029
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 3603.61
Epoch 1, Validation Loss: 4.14
Epoch 2, Validation Loss: -2.77
Epoch 3, Validation Loss: -5.74
Epoch 4, Validation Loss: -7.80
Epoch 5, Validation Loss: -6.14
Epoch 6, Validation Loss: -8.47
Epoch 7, Validation Loss: -8.45
Epoch 8, Validation Loss: -8.88
Epoch 9, Validation Loss: -8.79
Epoch 10, Validation Loss: -9.00
Epoch 11, Validation Loss: -8.97
Epoch 12, Validation Loss: -9.72
Epoch 13, Validation Loss: -9.65
Epoch 14, Validation Loss: -10.57
Epoch 15, Validation Loss: -9.88
Epoch 16, Validation Loss: -10.10
Epoch 17, Validation Loss: -10.83
Epoch 18, Validation Loss: -10.12
Epoch 19, Validation Loss: -9.80
Epoch 20, Validation Loss: -9.99
Epoch 21, Validation Loss: -11.34
Epoch 22, Validation Loss: -11.28
Epoch 23, Validation Loss: -9.82
Epoch 24, Validation Loss: -11.55
Epoch 25, Validation Loss: -10.97
Epoch 26, Validation Loss: -11.75
Epoch 27, Validation Loss: -11.80
Epoch 28, Validation Loss: -11.58
Epoch 29, Validation Loss: -11.23
Epoch 3

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 29.32it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -13.589329          0.001726      0.003131
std      0.062509          0.009155      0.000742
min    -13.825783          0.000000      0.001993
25%    -13.602283          0.000000      0.002636
50%    -13.586963          0.000000      0.003005
75%    -13.546951          0.000000      0.003513
max    -13.459233          0.065940      0.006034
Number of infeasible solution: 4


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2485.01
Epoch 1, Validation Loss: 3.37
Epoch 2, Validation Loss: -6.44
Epoch 3, Validation Loss: -8.40
Epoch 4, Validation Loss: -9.23
Epoch 5, Validation Loss: -9.12
Epoch 6, Validation Loss: -10.19
Epoch 7, Validation Loss: -9.11
Epoch 8, Validation Loss: -9.75
Epoch 9, Validation Loss: -8.82
Epoch 10, Validation Loss: -10.60
Epoch 11, Validation Loss: -11.07
Epoch 12, Validation Loss: -11.12
Epoch 13, Validation Loss: -11.44
Epoch 14, Validation Loss: -9.58
Epoch 15, Validation Loss: -11.65
Epoch 16, Validation Loss: -11.15
Epoch 17, Validation Loss: -8.22
Epoch 18, Validation Loss: -8.99
Epoch 19, Validation Loss: -11.03
Epoch 20, Validation Loss: -11.27
Epoch 21, Validation Loss: -10.81
Epoch 22, Validation Loss: -10.77
Epoch 23, Validation Loss: -11.66
Epoch 24, Validation Loss: -11.35
Epoch 25, Validation Loss: -11.87
Epoch 26, Validation Loss: -11.33
Epoch 27, Validation Loss: -12.19
Epoch 28, Validation Loss: -10.84
Epoch 29, Validation Loss: -11.80
E

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 27.78it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -13.744608          0.003400      0.003499
std      0.280072          0.017737      0.001164
min    -14.240952          0.000000      0.001990
25%    -13.927554          0.000000      0.002999
50%    -13.747429          0.000000      0.003006
75%    -13.610425          0.000000      0.003998
max    -12.891432          0.150890      0.007123
Number of infeasible solution: 7


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 256           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2482.40
Epoch 1, Validation Loss: -5.52
Epoch 2, Validation Loss: -10.61
Epoch 3, Validation Loss: -11.78
Epoch 4, Validation Loss: -11.27
Epoch 5, Validation Loss: -12.69
Epoch 6, Validation Loss: -12.93
Epoch 7, Validation Loss: -12.88
Epoch 8, Validation Loss: -12.59
Epoch 9, Validation Loss: -13.56
Epoch 10, Validation Loss: -14.27
Epoch 11, Validation Loss: -13.90
Epoch 12, Validation Loss: -13.50
Epoch 13, Validation Loss: -14.48
Epoch 14, Validation Loss: -13.69
Epoch 15, Validation Loss: -13.29
Epoch 16, Validation Loss: -14.16
Epoch 17, Validation Loss: -13.76
Epoch 18, Validation Loss: -14.54
Epoch 19, Validation Loss: -15.16
Epoch 20, Validation Loss: -14.37
Epoch 21, Validation Loss: -14.57
Epoch 22, Validation Loss: -14.43
Epoch 23, Validation Loss: -14.97
Epoch 24, Validation Loss: -14.23
Epoch 25, Validation Loss: -14.97
Epoch 26, Validation Loss: -14.43
Epoch 27, Validation Loss: -15.15
Epoch 28, Validation Loss: -15.41
Epoch 29, Validation Los

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_pr_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 25.81it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -17.421335          0.201050      0.000788
std      0.250103          0.262282      0.000617
min    -17.879455          0.000000      0.000000
25%    -17.599104          0.006625      0.000000
50%    -17.429028          0.098692      0.001000
75%    -17.257874          0.287999      0.001016
max    -16.819245          1.256986      0.003000
Number of infeasible solution: 78


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2485.01
Epoch 1, Validation Loss: 49.42
Epoch 2, Validation Loss: 4.36
Epoch 3, Validation Loss: 1.43
Epoch 4, Validation Loss: -1.41
Epoch 5, Validation Loss: -3.71
Epoch 6, Validation Loss: -6.00
Epoch 7, Validation Loss: -7.08
Epoch 8, Validation Loss: -8.11
Epoch 9, Validation Loss: -9.06
Epoch 10, Validation Loss: -9.43
Epoch 11, Validation Loss: -9.25
Epoch 12, Validation Loss: -9.97
Epoch 13, Validation Loss: -9.17
Epoch 14, Validation Loss: -10.31
Epoch 15, Validation Loss: -10.05
Epoch 16, Validation Loss: -10.12
Epoch 17, Validation Loss: -10.79
Epoch 18, Validation Loss: -10.45
Epoch 19, Validation Loss: -10.87
Epoch 20, Validation Loss: -10.40
Epoch 21, Validation Loss: -10.50
Epoch 22, Validation Loss: -10.22
Epoch 23, Validation Loss: -10.52
Epoch 24, Validation Loss: -10.54
Epoch 25, Validation Loss: -9.30
Epoch 26, Validation Loss: -10.27
Epoch 27, Validation Loss: -10.66
Epoch 28, Validation Loss: -10.10
Epoch 29, Validation Loss: -10.52
Epoch

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_st_100-100.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.26it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -10.992014               0.0      0.002165
std      0.480707               0.0      0.000670
min    -11.832628               0.0      0.000994
25%    -11.259485               0.0      0.001999
50%    -11.066118               0.0      0.002005
75%    -10.695165               0.0      0.002628
max     -9.085577               0.0      0.003584
Number of infeasible solution: 0
